**Import the necessary libraries**

In [52]:
import numpy as np
import pandas as pd

from datetime import date

**Load the data using pandas read_csv function. The data has already been pre-processed and cleaned so no EDA or preprocessing necessary**

In [53]:
df = pd.read_csv('/Users/thomasz/Desktop/subscription_contract_data.csv'
                 , parse_dates=['contract_sale_date', 'contract_churn_date'])

**Data field descriptions**

- contract_id - The unique id of each contract in the system
- contract_sale_date - The date each contract was sold 
- contract_churn_date - The date each contract was churned. A null value means the contract is still alive.

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 3 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   contract_id          200000 non-null  float64       
 1   contract_sale_date   200000 non-null  datetime64[ns]
 2   contract_churn_date  61564 non-null   datetime64[ns]
dtypes: datetime64[ns](2), float64(1)
memory usage: 4.6 MB


In [55]:
import warnings
warnings.filterwarnings('ignore')

**Create an is_churned column to flag contracts that have churned and those that are live based on whether the contract_churn_date column is null**

In [56]:
df.insert(0, "is_churned", np.where(df['contract_churn_date'].isna(), 0, 1))

**Set the live contracts to churn as of the date the report is run and calculate the difference between contract_sale_date and contract_churn_date in whole months (round down)**

In [57]:
todays_date = pd.Timestamp(date.today())

df['policy_length_months'] = np.where(df['contract_churn_date'].isna(), 
                                     ((todays_date - df.contract_sale_date)/np.timedelta64(1, 'M')),
                                     ((df.contract_churn_date - df.contract_sale_date)/np.timedelta64(1, 'M')) 
                                     )

df['policy_length_months'] = df['policy_length_months'].astype(int)

**Build a dataframe to calculate the number of contracts churned by month churned. Live contracts are being "churned" at the latest month that they were live and included**

In [58]:
df_churned_by_month = df[['policy_length_months', 'contract_id', 'is_churned']]

# Aggregate (count) the dataset by number of contracts churned by length in months
df_churned_by_month = \
df_churned_by_month.groupby('policy_length_months')['contract_id'].count().reset_index()
df_churned_by_month.rename(columns={"contract_id": "no_contracts_churned"}, inplace=True)

Build a survivorship table as per this Life Time table -> https://en.wikipedia.org/wiki/Life_table

**Create the contracts at start of period**

In [59]:
df_churned_by_month['contracts_at_start_of_period'] = df_churned_by_month['no_contracts_churned'].sum()

for i in range(0, len(df_churned_by_month.no_contracts_churned)):
    if i != 0:
        df_churned_by_month['contracts_at_start_of_period'][i] = \
        df_churned_by_month['contracts_at_start_of_period'][i-1].copy() - \
        df_churned_by_month['no_contracts_churned'][i-1].copy()
        
df_churned_by_month = df_churned_by_month.append({'policy_length_months':max(df_churned_by_month.policy_length_months) + 1, 
                            'no_contracts_churned':0,
                            'contracts_at_start_of_period':0}, ignore_index=True)

**Create the survivorship_rate column**

In [60]:
df_churned_by_month['survivorship_rate'] = 0.0000

for i in range(0, len(df_churned_by_month.survivorship_rate)):
    df_churned_by_month['survivorship_rate'][i] = \
    df_churned_by_month['contracts_at_start_of_period'][i].copy() / \
    df_churned_by_month['contracts_at_start_of_period'][0].copy()

**Create the avg_contract_months column**

In [61]:
df_churned_by_month['avg_contract_months'] = 0

for i in range(0, len(df_churned_by_month.avg_contract_months)):
    if i != len(df_churned_by_month.avg_contract_months)-1:
        df_churned_by_month['avg_contract_months'][i] = \
        (df_churned_by_month['contracts_at_start_of_period'][i].copy() + \
         df_churned_by_month['contracts_at_start_of_period'][i+1].copy()) / 2

**Create the expected_contract_months column**

In [62]:
df_churned_by_month['expected_contract_months'] = 0

for i in range(0, len(df_churned_by_month.expected_contract_months)):
    df_churned_by_month['expected_contract_months'][i] = \
    df_churned_by_month['avg_contract_months'][i:len(df_churned_by_month.expected_contract_months)].sum()

**Create the expected_contract_months_unit column**

In [63]:
df_churned_by_month['expected_contract_months_unit'] = 0.0000

for i in range(0, len(df_churned_by_month.expected_contract_months)):
    df_churned_by_month['expected_contract_months_unit'][i] = \
    df_churned_by_month['expected_contract_months'][i].copy() / \
    df_churned_by_month['contracts_at_start_of_period'][i].copy()

In [64]:
# df_churned_by_month.to_excel("survivorship_table.xlsx")

In [65]:
df_churned_by_month

,policy_length_months,no_contracts_churned,contracts_at_start_of_period,survivorship_rate,avg_contract_months,expected_contract_months,expected_contract_months_unit
0,0,8167,200000,1.000000,195916,2287858,11.439290
1,1,10455,191833,0.959165,186605,2091942,10.905016
2,2,11480,181378,0.906890,175638,1905337,10.504786
3,3,11389,169898,0.849490,164203,1729699,10.180808
4,4,12461,158509,0.792545,152278,1565496,9.876386
5,5,11017,146048,0.730240,140539,1413218,9.676394
6,6,8572,135031,0.675155,130745,1272679,9.425088
7,7,8799,126459,0.632295,122059,1141934,9.030073
8,8,10393,117660,0.588300,112463,1019875,8.667984
9,9,8188,107267,0.536335,103173,907412,8.459377
